# Using the LI API to triage and price a set of locations

As an insurance underwriter, a set of 239 commercial locations from the southeast US (Texas, the Gulf Coast, Florida, and South Carolina) has come across your desk. You need to triage these locations: which should be removed because they don't pass your underwriting guidelines, and for those that remain, what is the cat premium?

## Setup

In [ ]:
# Make sure you have Python 3, and that at least requests 2.20 and pandas 0.23 are installed
import sys

if (sys.version_info < (3, 0)):
    print("This notebook only works with Python 3 and later")
else:
    !{sys.executable} -m pip install "requests>=2.20"
    !{sys.executable} -m pip install "pandas>=0.23"

In [2]:
# Configure Pandas
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
import json
import requests
import logging
import threading
from multiprocessing.pool import ThreadPool

local = threading.local()

def getRowValue(column, row):
    if column in row:
        return row[column]
    else:
        return ''


def geocode(row):
    try:
        # Session per thread for connection re-use
        if not hasattr(local, 'session'):
            local.session = requests.Session()

        headers = {
            'content-type': 'application/json',
            'authorization': YOUR_API_KEY
        }

        url = API_URL + '/geocode/latest'

        request = {
            "admin1Code": row['admin1Code'],
            "countryCode": row['countryCode'],
            "countryScheme": row['countryScheme'],
            "postalCode": row['postalCode'],
            "streetAddress": row['streetAddress']
        }

        response = local.session.post(url, headers=headers,
                                      data=json.dumps(request))
        response_json = json.loads(response.text)
        row['geocode.json'] = response_json
        for key, value in response_json.items():
            row['geocode.' + str(key)] = value
        if response.status_code >= 400 and response.status_code < 500:
            logging.warning('Unable to geocode row id=' +
                            str(row['id']) + ': ' + str(getRowValue('geocode.errors', row)))
        elif response.status_code >= 500:
            logging.error('Error during geocode of row id=' +
                          str(row['id']) + ': ' + str(getRowValue('geocode.errors', row)))
        return row

    except Exception as e:
        raise e


def lookup(row, layers):
    for layer in layers:
        try:
            # Session per thread for connection re-use
            if not hasattr(local, 'session'):
                local.session = requests.Session()

            headers = {
                'content-type': 'application/json',
                'authorization': YOUR_API_KEY
            }

            url = API_URL + '/' + layer + '/latest'

            request = {
                'location': {
                    'address': row['geocode.json'],
                    'characteristics': {
                        'construction': str(getRowValue('construction', row)),
                        'occupancy': str(getRowValue('occupancy', row)),
                        'yearBuilt': str(getRowValue('yearBuilt', row)),
                        'numOfStories': str(getRowValue('numOfStories', row))
                    },
                    'coverageValues': {}
                }
            }

            response = local.session.post(
                url, headers=headers, data=json.dumps(request))
            response_json = json.loads(response.text)
            for key, value in response_json.items():
                row[layer + '.' + str(key)] = value
            if response.status_code >= 400 and response.status_code < 500:
                logging.warning('Unable to lookup(' + layer + ') row id=' +
                                str(row['id']) + ': ' + str(getRowValue(layer + '.errors', row)))
            elif response.status_code >= 500:
                logging.error('Error during lookup(' + layer + ') of row id=' +
                              str(row['id']) + ': ' + str(getRowValue(layer + '.errors', row)))

        except Exception as e:
            raise e

    return row


# Workaround for apply being performed twice on the first row (see pandas apply documentation)
def apply_workaround_df(row, funct, counter, **kwargs):
    if(counter['count'] == 0):
        # Trigger the rigt codepath
        row['trigger'] = 1
        counter['count'] = counter['count'] + 1
        return row
    else:
        counter['count'] = counter['count'] + 1
        return funct(row, **kwargs)


def apply_df(args):
    df, func, num, kwargs = args
    counter = {'count': 0}
    return num, df.apply(apply_workaround_df, funct=func, counter=counter, **kwargs)


def parallel_apply_df(self, func, **kwargs):
    threads = kwargs.pop('threads')
    pool = ThreadPool(threads)
    result = pool.map(apply_df, [(d, func, i, kwargs)
                                 for i, d in enumerate(np.array_split(self, threads))])
    pool.close()
    result = sorted(result, key=lambda x: x[0])
    return pd.concat([i[1] for i in result], sort=True)


pd.DataFrame.parallel_apply = parallel_apply_df

## Set environment variables

In [4]:
API_URL = 'https://api.rms.com/li'
YOUR_API_KEY = 'insert your key here'

### Load the locations, also known as the schedule of values

In [5]:
data = pd.read_csv('data.csv', dtype={
                   'postalCode': object, 'occType': object, 'bldgClass': object}).fillna('')

# Create columns for occupancy and construction by joining the scheme and type
data['occupancy'] = data[['occScheme', 'occType']].apply(
    lambda x: ''.join(x), axis=1)
data['construction'] = data[['bldgScheme', 'bldgClass']].apply(
    lambda x: ''.join(x), axis=1)

data

id                                               streetAddress  \
0    1    770 KEMP BOTTOM RD                                           
1    2    2001 KINGSLEY AVE                                            
2    3    1050 COUNTRY LN                                              
3    4    6675 US HIGHWAY 43                                           
4    5    844 ENGLAND ST                                               
5    6    5775 PEACHTREE DUNWOODY RD NE                                
6    7    13001 SOUTHERN BLVD                                          
7    8    8201 W BROWARD BLVD                                          
8    9    660 W SAVANNAH CIR                                           
9    10   1918 UNIVERSITY BLVD                                         
10   11   1400 STATE DOCKS RD                                          
11   12   1792 ANDREW JACKSON CT                                       
12   13   8605 LARGO LAKES DR                                          
13   14   4955 MASON RD                                                
14   15   72 MARIETTA ST NW                                            
15   16   5353 S STATE ROAD 7                                          
16   17   2450 GALLERIA PKWY SE                                        
17   18   1183 COUNTY ROAD                                             
18   19   5637 MARINE PKWY                                             
19   20   13615 OLD HIGHWAY 61 N                                       
20   21   3704 LOUIS RICH RD                                           
21   22   400 S BUNCOMBE RD                                            
22   23   2751 S DIXIE HWY                                             
23   24   5301 S CONGRESS AVE                                          
24   25   300 NISSAN DR                                                
25   26   COPE STEAM ELECTRIC PLANT                                    
26   27   1 PLASTICS DR                                                
27   28   1050 VERN CORA RD                                            
28   29   271 WORLD DR                                                 
29   30   148 INDUSTRIAL BLVD                                          
30   31   280 BEACH BLVD                                               
31   32   3602 PORT SUTTON RD                                          
32   33   60 HERRING RD                                                
33   34   401 NW 42ND AVE                                              
34   35   188 HALEY AVE BLDG 1                                         
35   36   100 BEST BUY DRIVE                                           
36   37   T0501 MAIN CAMPUS                                            
37   38                                                                
38   39   1390 MAGIC KINGDOM DR                                        
39   40   5300 CURETON FERRY RD                                        
40   41   ATHENS HWY AT MINERAL SPRINGS RD                             
41   42   3434 PEACHTREE RD NE                                         
42   43   40 PERIMETER CTR E                                           
43   44   3001 W DR MARTIN LUTHER KING JR BLVD                         
44   45   1567 LUKKEN INDUSTRIAL DR W                                  
45   46   3464 MIKE PADGETT HWY                                        
46   47   2201 ORLEANS DR.                                             
47   48   1400 MURPHY AVE SW                                           
48   49   1700 CENTER ST                                               
49   50   4224 METRIC DR                                               
50   51   1 E TELECOM PKWY                                             
51   52   5700 VALLEY RD                                               
52   53   5223 ORIENT RD                                               
53   54   4000 VILLAGE VIEW DRIVE                                      
54   55   3900 MOTORS INDUSTRIAL WAY          

### Geocode the locations
Enhance your addresses with their coordinates and other address info.

In [6]:
geocoded = data.parallel_apply(geocode, threads=10, axis=1)
geocoded.drop(['geocode.json'], axis=1)

admin1Code bldgClass bldgScheme  buildingValue  businessInterruptionValue  \
0    MS         0         ATC        9.464895e+08   0.000000e+00                
1    FL         3         ATC        3.549898e+07   0.000000e+00                
2    FL         0         ATC        2.747445e+08   9.070515e+07                
3    AL         9         ATC        1.627591e+08   0.000000e+00                
4    FL         1         ATC        9.004787e+07   0.000000e+00                
5    GA         0         ATC        1.793077e+08   0.000000e+00                
6    FL         3         ATC        3.457876e+07   0.000000e+00                
7    FL         3         ATC        3.616376e+07   0.000000e+00                
8    FL         0         ATC        3.242413e+08   1.503256e+08                
9    AL         6         ATC        1.003732e+08   0.000000e+00                
10   AL         4         ATC        3.459918e+08   0.000000e+00                
11   FL         1         ATC        6.798017e+07   0.000000e+00                
12   FL         0         ATC        9.790293e+06   0.000000e+00                
13   GA         9         ATC        0.000000e+00   2.688914e+08                
14   GA         6         ATC        2.278427e+08   4.339996e+07                
15   FL         45        RMS        4.602902e+07   1.244886e+07                
16   GA         3         ATC        5.161232e+07   0.000000e+00                
17   MS         0         ATC        3.025000e+08   0.000000e+00                
18   FL         1         ATC        4.325983e+07   0.000000e+00                
19   MS         0         ATC        5.184608e+08   1.686762e+08                
20   SC         10        ATC        6.868840e+07   0.000000e+00                
21   SC         3         ATC        2.553896e+07   0.000000e+00                
22   FL         0         ATC        1.017723e+08   0.000000e+00                
23   FL         3         ATC        1.364912e+08   0.000000e+00                
24   MS         2         ATC        5.264095e+08   0.000000e+00                
25   SC         9         ATC        7.193873e+08   0.000000e+00                
26   AL         9         ATC        5.981645e+07   3.517842e+08                
27   SC         4         RMS        6.600218e+07   0.000000e+00                
28   FL         0         ATC        3.242413e+08   1.503256e+08                
29   GA         0         ATC        1.372800e+07   0.000000e+00                
30   MS         0         ATC        1.232000e+08   1.094401e+08                
31   FL         0         ATC        1.021491e+09   0.000000e+00                
32   GA         9         ATC        1.951986e+08   0.000000e+00                
33   FL         3         ATC        5.676000e+07   0.000000e+00                
34   GA         1         ATC        5.059822e+07   0.000000e+00                
35   GA         0         ATC        5.005000e+07   0.000000e+00                
36   AL         6         ATC        1.321447e+08   0.000000e+00                
37   FL         9         ATC        6.036140e+07   0.000000e+00                
38   FL         0         ATC        3.242413e+08   1.503256e+08                
39   SC         0         ATC        1.751024e+08   0.000000e+00                
40   GA         3         ATC        1.133679e+07   0.000000e+00                
41   GA         0         ATC        2.883187e+07   3.410591e+07                
42   GA         0         ATC        1.806869e+07   0.000000e+00                
43   FL         5         FIRE       2.979077e+08   0.000000e+00                
44   GA         0         RMS        5.936509e+07   0.000000e+00                
45   GA         4A        RMS        1.055395e+08   0.000000e+00                
46   FL         0         ATC        3.242413e+08   1.503256e+08                
47   GA         9         ATC        4.184840e+07   0.000000e+00                
48   AL         9         ATC      

## Data Triage
### Remove locations with poor geocoding
Your company has an underwriting policy that it does not insure locations that geocode to postcalcode-or-worse resolution. In other words, it does not want to insure locations unless it has a good idea where those locations actually are. Remove the offending locations from your data set.

In [7]:
geocodedFiltered = geocoded[geocoded['geocode.rmsGeoModelResolutionCode'] < 5].reset_index(drop=True)
print("Removed " + str(geocoded.shape[0] -
                       geocodedFiltered.shape[0]) + " rows")

Removed 33 rows


### Remove locations in flood zones
Your company has an underwriting policy that it does not insure locations in FEMA 100-year flood zones, which are those that begin with an "A": AO, AE, A19, etc. Look up the FEMA zones and remove the offending locations from your data set.

In [8]:
geocodedFilteredWithFema = geocodedFiltered.parallel_apply(lookup, threads=10, layers=['us_fema_firm'], axis=1)
geocodedFilteredWithFema[['id', 'us_fema_firm.panel', 'us_fema_firm.community', 'us_fema_firm.floodZone', 'us_fema_firm.sfha', 'us_fema_firm.panelDate', 'us_fema_firm.bfe',
                          'us_fema_firm.annualProbability', 'us_fema_firm.levees', 'us_fema_firm.otherZones', 'us_fema_firm.flMatch', 'us_fema_firm.floodway', 'us_fema_firm.cobra']]


id us_fema_firm.panel us_fema_firm.community us_fema_firm.floodZone  \
0    1    28149C0291D        280176                 SHX                     
1    2    12019C0090E        120064                 X                       
2    3    12095C0245F        120186                 X                       
3    4    01093C0384C        010162                 SHX                     
4    5    12031C0244J        120077                 AE                      
5    6    13121C0161F        130669                 X                       
6    7    12099C0542F        120192                 AE                      
7    8    12011C0535H        120054                 SHX                     
8    9    12095C0580F        120576                 D                       
9    11   01103C0018F        010175                 X                       
10   12   12033C0531G        120080                 X                       
11   13   12103C0119G        125139                 X                       
12   14   13121C0456F        130316                 X                       
13   15   13121C0244F        135157                 X                       
14   16   12011C0562H        120035                 SHX                     
15   17   13067C0139H        130052                 X                       
16   19   12101C0351F        120232                 SHX                     
17   20   28143C0085D        280236                 SHX                     
18   21   45071C0217C        450224                 X                       
19   22   45045C0342E        450089                 X                       
20   23   12099C0583F        120229                 X                       
21   24   12099C0777F        120193                 X                       
22   25   28089C0405F        280228                 X                       
23   27   01085C0135D        010272                 X                       
24   28   45059C0215C        450122                 X                       
25   29   12095C0390F        120576                 D                       
26   30   13237C0161C        130218                 X                       
27   31   28047C0313G        285252                 VE                      
28   32   12057C0366H        120112                 AE                      
29   33   13077C0142D        130298                 X                       
30   34   12011C0364H        120054                 X                       
31   35   13051C0145G        135163                 X                       
32   36   13175C0189C        130217                 X                       
33   39   12095C0390F        120576                 D                       
34   40   45091C0485E        450193                 X                       
35   41   13105C0210C        135264                 X                       
36   42   13121C0251F        135157                 X                       
37   43   13089C0012J        130679                 X                       
38   44   12057C0351H        120114                 X                       
39   45   13285C0143E        130177                 X                       
40   46   13245C0140G        130158                 X                       
41   47   12095C0395F        120341                 D                       
42   48   13121C0358F        135157                 X                       
43   49   01097C0554K        015007                 X                       
44   50   12095C0260F        120179                 X                       
45   51   12057C0230H        120115                 X                       
46   52   01073C0531G        010217                 X                       
47   53   12057C0380J        120112                 X                       
48   54   13139C0088G        130466                 X                       
49   55   13089C0018J        130069                 X                       
50   56   13121C0251F        135157                 X                       
51

In [9]:
notInFloodZones = geocodedFilteredWithFema[~geocodedFilteredWithFema['us_fema_firm.floodZone'].str.startswith('A')].reset_index(drop=True)
print("Removed " +
      str(geocodedFilteredWithFema.shape[0] - notInFloodZones.shape[0]) + " rows")

Removed 22 rows


### Examine RMS modeled flood depths
In 2017, your portfolio was badly hit by Hurricane Harvey, even though it tried to avoid risky locations in flood zones. As a consequence, you don't want to only rely on FEMA for your lookups. Look up the RMS modeled flood depths -- combined inland flood and surge, for the 100-year return period -- and remove any location with more than 2 feet of flooding (depth score > 3) from your data set.

In [10]:
notInFloodZonesWithDepths = notInFloodZones.parallel_apply(lookup, threads=10, layers=['us_fl_depth'], axis=1)
notInFloodZonesWithDepths.loc[:, notInFloodZonesWithDepths.columns.str.startswith('us_fl_depth')]


us_fl_depth.depthCombinedDef1000yr  us_fl_depth.depthCombinedDef100yr  \
0    9                                   0                                   
1    0                                   0                                   
2    1                                   0                                   
3    0                                   0                                   
4    0                                   0                                   
5    0                                   0                                   
6    1                                   0                                   
7    0                                   0                                   
8    1                                   1                                   
9    0                                   0                                   
10   0                                   0                                   
11   0                                   0                                   
12   0                                   0                                   
13   0                                   0                                   
14   14                                  0                                   
15   0                                   0                                   
16   0                                   0                                   
17   0                                   0                                   
18   0                                   0                                   
19   0                                   0                                   
20   0                                   0                                   
21   0                                   0                                   
22   0                                   0                                   
23   1                                   1                                   
24   0                                   0                                   
25   14                                  14                                  
26   0                                   0                                   
27   0                                   0                                   
28   0                                   0                                   
29   0                                   0                                   
30   3                                   3                                   
31   1                                   1                                   
32   0                                   0                                   
33   0                                   0                                   
34   0                                   0                                   
35   0                                   0                                   
36   0                                   0                                   
37   0                                   0                                   
38   1                                   1                                   
39   0                                   0                                   
40   0                                   0                                   
41   2                                   2                                   
42   0                                   0                                   
43   4                                   4                                   
44   0                                   0                                   
45   0                                   0                                   
46   0                                   0                                   
47   0                                   0                                   
48   0                                   0                                   
49   0                                   0                                   
50   0                     

In [11]:
notFlooded = notInFloodZonesWithDepths[notInFloodZonesWithDepths['us_fl_depth.depthCombinedDef100yr'] <= 3].reset_index(drop=True)
print("Removed " + str(notInFloodZonesWithDepths.shape[0] - notFlooded.shape[0]) + " rows")

Removed 11 rows


# Data Enhancement
You're now down to 173 locations. Before retreiving scores and loss costs for the data, you want to ensure that you're using the best input data possible. For that, you turn to the ExposureSource Database. For each location, you return additional information about its primary attributes and secondary modifiers. Then you enhance your own data set with this information, replacing your unknown and known values with those in the database. (Note: you could also choose to preserve your values and only fill in values where yours were unknown.)

In [12]:
enhanced = notFlooded.parallel_apply(lookup, threads=10, layers=['us_esdb_building_attributes'], axis=1).fillna(
    value={'us_esdb_building_attributes.yearBuilt': 0, 'us_esdb_building_attributes.numOfStories': 0,
           'us_esdb_building_attributes.occupancy': '', 'us_esdb_building_attributes.construction': ''}).astype(
               {'us_esdb_building_attributes.yearBuilt': int, 'us_esdb_building_attributes.numOfStories': int})
enhanced[['yearBuilt', 'us_esdb_building_attributes.yearBuilt',
          'numOfStories', 'us_esdb_building_attributes.numOfStories']]


yearBuilt  us_esdb_building_attributes.yearBuilt  numOfStories  \
0    0          0                                      0              
1    1974       1974                                   4              
2    0          0                                      0              
3    1954       1954                                   1              
4    1979       1979                                   0              
5    1975       1975                                   3              
6    0          0                                      0              
7    1970       1970                                   1              
8    1900       1900                                   2              
9    1989       1989                                   1              
10   2000       2000                                   1              
11   1971       1971                                   8              
12   0          0                                      0              
13   1903       1903                                   14             
14   1971       1971                                   2              
15   0          0                                      0              
16   0          0                                      0              
17   1976       1976                                   0              
18   0          0                                      0              
19   1965       1965                                   6              
20   2003       2003                                   1              
21   1987       1987                                   4              
22   1988       1988                                   2              
23   0          0                                      0              
24   1968       1968                                   0              
25   1999       1999                                   3              
26   1969       1969                                   3              
27   1953       1953                                   2              
28   0          0                                      0              
29   0          0                                      0              
30   0          0                                      0              
31   0          0                                      0              
32   1983       1983                                   22             
33   1977       1977                                   2              
34   1964       1964                                   8              
35   0          0                                      0              
36   1963       1963                                   1              
37   0          0                                      0              
38   1938       1938                                   0              
39   2006       2006                                   3              
40   1984       1984                                   1              
41   1986       1986                                   3              
42   2004       2004                                   14             
43   2001       2001                                   6              
44   1940       1940                                   1              
45   1959       1959                                   4              
46   0          0                                      0              
47   1975       1975                                   1              
48   2007       2007                                   5              
49   1968       1968                                   1              
50   1996       1996                                   24             
51   0          2002                                   0              
52   0          0                                      0              
53   1990       1990                                   5              
54   0          0                                      0              
55   0          0            

In [13]:
def blendCharacteristicsInt(row, esdbLabel, myLabel):
    esdbValue = row[esdbLabel]
    myValue = row[myLabel]
    if(esdbValue > 0):
        row[myLabel] = esdbValue
    else:
        row[myLabel] = myValue
    return row


def blendCharacteristicsString(row, esdbLabel, myLabel):
    esdbValue = row[esdbLabel]
    myValue = row[myLabel]
    if(esdbValue != None and esdbValue != ''):
        row[myLabel] = esdbValue
    else:
        row[myLabel] = myValue
    return row


blended = enhanced \
    .apply(blendCharacteristicsInt, axis=1, esdbLabel='us_esdb_building_attributes.yearBuilt', myLabel='yearBuilt') \
    .apply(blendCharacteristicsInt, axis=1, esdbLabel='us_esdb_building_attributes.numOfStories', myLabel='numOfStories') \
    .apply(blendCharacteristicsString, axis=1, esdbLabel='us_esdb_building_attributes.occupancy', myLabel='occupancy') \
    .apply(blendCharacteristicsString, axis=1, esdbLabel='us_esdb_building_attributes.construction', myLabel='construction')

blended[['yearBuilt', 'numOfStories', 'occupancy', 'construction']]

yearBuilt  numOfStories occupancy construction
0    0          0             ATC30     ATC0       
1    1974       4             ATC9      ATC3       
2    0          0             ATC3      ATC0       
3    1954       1             ATC12     ATC9       
4    1979       6             ATC8      ATC0       
5    1975       3             ATC9      ATC3       
6    0          0             ATC10     ATC0       
7    1970       1             ATC15     ATC4       
8    1900       2             ATC4      ATC1       
9    1989       1             ATC8      ATC0       
10   2000       1             ATC12     ATC9       
11   1971       8             ATC8      ATC6       
12   0          0             ATC0      RMS45      
13   1903       14            ATC3      ATC3       
14   1971       2             ATC9      ATC1       
15   0          0             ATC10     ATC0       
16   0          0             ATC14     ATC10      
17   1976       0             ATC38     ATC3       
18   0          0             ATC8      ATC0       
19   1965       6             ATC9      ATC3       
20   2003       1             ATC12     ATC2       
21   1987       4             ATC12     ATC9       
22   1988       2             ATC6      RMS4       
23   0          0             ATC10     ATC0       
24   1968       0             ATC5      ATC0       
25   1999       3             ATC6      ATC9       
26   1969       3             ATC9      ATC3       
27   1953       2             ATC4      ATC1       
28   0          0             ATC5      ATC0       
29   0          0             ATC10     ATC0       
30   0          0             ATC12     ATC0       
31   0          0             ATC38     ATC3       
32   1983       22            ATC3      ATC0       
33   1977       2             ATC14     ATC0       
34   1964       8             ATC9      FIRE5      
35   0          0             ATC15     RMS0       
36   1963       1             ATC15     RMS4A      
37   0          0             ATC3      ATC0       
38   1938       0             ATC14     ATC9       
39   2006       3             ATC19     ATC9       
40   1984       1             ATC6      ATC0       
41   1986       3             ATC8      RMS3A      
42   2004       14            ATC3      ATC7       
43   2001       6             ATC4      ATC10      
44   1940       1             ATC12     ATC9       
45   1959       4             ATC5      ATC6       
46   0          0             ATC13     ATC0       
47   1975       1             ATC6      ATC16      
48   2007       5             ATC4      RMS6       
49   1968       1             ATC14     ATC0       
50   1996       24            ATC23     ATC6       
51   2002       16            ATC8      ATC0       
52   0          0             ATC3      ATC0       
53   1990       5             ATC3      ATC0       
54   0          0             ATC3      ATC0       
55   0          0             ATC15     ATC0       
56   0          0             ATC16     ATC0       
57   1973       4             ATC9      ATC3       
58   1980       4             ATC11     ATC6       
59   1954       1             ATC34     ATC9       
60   1995       1             ATC37     ATC10      
61   0          0             ATC3      ATC0       
62   1972       5             ATC9      ATC2       
63   1994       4             ATC9      ATC3       
64   0          0             ATC5      ATC0       
65   1994       7             ATC3      ATC0       
66   1950       6             ATC9      FIRE5      
67   0          0             ATC7      ATC0       
68   1957       2             ATC4      ATC1       
69   2000       3             ATC10     ATC6       
70   1981       2             ATC8      ATC9       
71   0          0             ATC15     ATC0       
72   1975       3             ATC5      ATC0       
73   0          0             ATC15     ATC0       
74   1976       7             ATC9      ATC2       
75   1976       73            ATC3      ATC6       
7

## Data scoring
Now it's time to score your data. In the southeast, you need to think about several perils: hurricane, flooding, severe convective storm (tornado, hail, and straight-line wind), and even earthquake in South Carolina. While higher risk usually is paired with higher premium, you want to make sure that you're not insuring the very worst locations in the state. First you return the 100-year risk score for each location for all perils, plus, where available, its relative risk among other locations in its state ("percentileAdmin1").

In [14]:
riskScores = blended.parallel_apply(lookup, threads=10, layers=[
                                    'us_ws_risk_score', 'us_cs_risk_score', 'us_eq_risk_score'], axis=1)

riskScores[['us_ws_risk_score.percentileAdmin1',
            'us_cs_risk_score.percentileAdmin1', 
            'us_eq_risk_score.percentileAdmin1']]

us_ws_risk_score.percentileAdmin1  us_cs_risk_score.percentileAdmin1  \
0    35.907859                          11.904762                           
1    6.021005                           57.692307                           
2    24.834862                          39.423077                           
3    0.034048                           39.189190                           
4    0.033047                           69.465652                           
5    73.744238                          4.326923                            
6    26.445849                          57.692307                           
7    0.034048                           87.162163                           
8    50.121180                          98.557693                           
9    41.225110                          67.788460                           
10   0.033047                           45.801525                           
11   0.033047                           34.351147                           
12   91.892791                          11.538462                           
13   0.033047                           54.198475                           
14   39.286223                          87.500000                           
15   0.045167                           89.285713                           
16   0.029240                           75.000000                           
17   0.029240                           97.222221                           
18   90.861569                          4.807692                            
19   73.544647                          10.576923                           
20   35.546522                          89.285713                           
21   29.519918                          43.243244                           
22   0.029240                           91.666664                           
23   28.874210                          57.692307                           
24   0.033047                           35.114502                           
25   0.033047                           24.427481                           
26   73.359312                          4.326923                            
27   77.693325                          9.923664                            
28   23.859881                          31.297710                           
29   34.899967                          66.826920                           
30   0.029240                           72.222221                           
31   0.033047                           85.496185                           
32   0.033047                           14.503817                           
33   0.033047                           74.809158                           
34   33.954284                          45.192307                           
35   0.033047                           6.081081                            
36   30.204891                          66.666664                           
37   31.991636                          57.692307                           
38   0.033047                           43.511452                           
39   87.470208                          12.162162                           
40   25.466901                          55.769230                           
41   27.158675                          42.788460                           
42   33.531341                          66.826920                           
43   0.033047                           54.198475                           
44   0.033047                           63.358780                           
45   0.033047                           47.328243                           
46   0.033047                           94.444443                           
47   25.908853                          40.384617                           
48   81.737395                          12.019231                           
49   3.858765                           55.769230                           
50   27.579162                          76.351349                           
5

### Remove riskiest locations relative to their region
Remove locations where they are in the 90th percentile for state.

In [16]:
riskScoresFiltered = riskScores[(riskScores['us_ws_risk_score.percentileAdmin1'] < 90) &
                                (riskScores['us_cs_risk_score.percentileAdmin1'] < 90) &
                                (riskScores['us_eq_risk_score.percentileAdmin1'] < 90)
                                ].reset_index(drop=True)

print("Removed " + str(riskScores.shape[0] -
                       riskScoresFiltered.shape[0]) + " rows")

Removed 25 rows


#### Remove overall riskiest locations for flood
Because of the worries about flood, remove any flood location with a risk score of 5 or higher for this peril.

In [17]:
riskScoresWithFlood = riskScoresFiltered.parallel_apply(lookup, threads=10, layers=['us_fl_risk_score'], axis=1)
riskScoresWithFlood[['us_fl_risk_score.scoreOverall']]

us_fl_risk_score.scoreOverall
0    3                            
1    1                            
2    1                            
3    1                            
4    1                            
5    1                            
6    1                            
7    1                            
8    1                            
9    1                            
10   1                            
11   1                            
12   3                            
13   1                            
14   1                            
15   1                            
16   1                            
17   1                            
18   1                            
19   1                            
20   1                            
21   1                            
22   1                            
23   1                            
24   1                            
25   1                            
26   1                            
27   1                            
28   1                            
29   1                            
30   1                            
31   1                            
32   1                            
33   1                            
34   1                            
35   1                            
36   1                            
37   1                            
38   1                            
39   1                            
40   1                            
41   1                            
42   1                            
43   1                            
44   1                            
45   1                            
46   1                            
47   1                            
48   1                            
49   1                            
50   1                            
51   1                            
52   1                            
53   1                            
54   1                            
55   1                            
56   1                            
57   1                            
58   1                            
59   1                            
60   1                            
61   1                            
62   1                            
63   1                            
64   1                            
65   1                            
66   1                            
67   1                            
68   1                            
69   1                            
70   1                            
71   1                            
72   1                            
73   1                            
74   1                            
75   1                            
76   1                            
77   1                            
78   1                            
79   1                            
80   1                            
81   5                            
82   1                            
83   1                            
84   2                            
85   1                            
86   1                            
87   1                            
88   1                            
89   1                            
90   1                            
91   1                            
92   1                            
93   1                            
94   1                            
95   1                            
96   1                            
97   1                            
98   1                            
99   1                            
100  1                            
101  1                            
102  1                            
103  1                            
104  1                            
105  1                            
106  1                            
107  1                            
108  1                            
109  1                            
110  1                            
111  1                            
112  1                            
113  1         

In [19]:
riskScoresWithFloodFiltered = riskScoresWithFlood[riskScoresWithFlood['us_fl_risk_score.scoreOverall'] < 5].reset_index(drop=True)
print("Removed " + str(riskScoresWithFlood.shape[0] - riskScoresWithFloodFiltered.shape[0]) + " rows")


Removed 1 rows


## Data Pricing
You've gotten the data down to 147 locations. Now you're ready to submit them for pricing. You decide that you only need to factor in the cat premium if a location has a risk score of 4 or higher for a peril. Look up the loss costs for each location-peril combination where this is true.

Finally, calculate the cat premium:
1. Calculate the ground-up loss per peril: For each peril, multiply the coverage values by that coverage's annual loss rate.

2. Add up the ground-up loss across all perils. (Note: use 0 for perils that weren't looked up because their risk was low.)

3. To get the premium, multiply the ground-up loss by 1.5 to account for potential uncertainty in the modeled loss.

In [20]:
def conditionalLookup(row, layers):
    for layer in layers:
        if row[layer.replace('loss_cost', 'risk_score') + '.score100yr'] >= 4:
            lookup(row, [layer])
        else:
            row[layer + '.buildingAlr'] = 0
            row[layer + '.contentsAlr'] = 0
            row[layer + '.businessInterruptionAlr'] = 0
    return row

In [21]:
lossCosts = riskScoresWithFloodFiltered.parallel_apply(conditionalLookup, threads=10, layers=[
                                                       'us_ws_loss_cost', 'us_cs_loss_cost', 'us_eq_loss_cost'], axis=1)
lossCosts[['us_ws_loss_cost.buildingAlr',
           'us_cs_loss_cost.buildingAlr', 
           'us_eq_loss_cost.buildingAlr']]

us_ws_loss_cost.buildingAlr  us_cs_loss_cost.buildingAlr  \
0    0.000000                     0                             
1    0.000000                     0                             
2    0.000000                     0                             
3    0.000000                     0                             
4    0.000000                     0                             
5    0.006187                     0                             
6    0.000000                     0                             
7    0.000000                     0                             
8    0.000000                     0                             
9    0.000000                     0                             
10   0.000000                     0                             
11   0.000000                     0                             
12   0.000000                     0                             
13   0.000000                     0                             
14   0.000000                     0                             
15   0.005092                     0                             
16   0.000000                     0                             
17   0.000000                     0                             
18   0.000000                     0                             
19   0.000000                     0                             
20   0.000000                     0                             
21   0.006147                     0                             
22   0.000000                     0                             
23   0.000000                     0                             
24   0.000000                     0                             
25   0.000000                     0                             
26   0.000000                     0                             
27   0.000000                     0                             
28   0.000000                     0                             
29   0.000000                     0                             
30   0.000000                     0                             
31   0.000000                     0                             
32   0.000000                     0                             
33   0.000000                     0                             
34   0.000000                     0                             
35   0.000000                     0                             
36   0.000000                     0                             
37   0.000000                     0                             
38   0.000000                     0                             
39   0.000000                     0                             
40   0.000000                     0                             
41   0.000000                     0                             
42   0.000000                     0                             
43   0.000000                     0                             
44   0.000000                     0                             
45   0.000000                     0                             
46   0.000000                     0                             
47   0.000000                     0                             
48   0.000000                     0                             
49   0.000000                     0                             
50   0.000000                     0                             
51   0.000000                     0                             
52   0.000000                     0                             
53   0.000000                     0                             
54   0.000000                     0                             
55   0.000000                     0                             
56   0.000000                     0                             
57   0.000000                     0                             
58   0.000000                     0                             
59   0.000000                     0                             
60   0.000000                     0     

In [22]:
lossCostsWithFlood = lossCosts.parallel_apply(lookup, threads=10, layers=['us_fl_loss_cost'], axis=1)
lossCostsWithFlood[['us_fl_loss_cost.buildingAlr',
                    'us_fl_loss_cost.contentsAlr', 
                    'us_fl_loss_cost.businessInterruptionAlr']]

us_fl_loss_cost.buildingAlr  us_fl_loss_cost.contentsAlr  \
0    2.151194e-04                 2.939255e-04                  
1    0.000000e+00                 0.000000e+00                  
2    3.461000e-07                 1.337000e-06                  
3    9.500000e-09                 4.600000e-09                  
4    0.000000e+00                 0.000000e+00                  
5    1.219000e-07                 2.386000e-07                  
6    1.459000e-06                 4.394000e-06                  
7    0.000000e+00                 0.000000e+00                  
8    9.600000e-09                 2.030000e-08                  
9    0.000000e+00                 0.000000e+00                  
10   0.000000e+00                 0.000000e+00                  
11   0.000000e+00                 0.000000e+00                  
12   1.802391e-04                 2.523779e-04                  
13   4.910900e-06                 1.131490e-05                  
14   0.000000e+00                 0.000000e+00                  
15   0.000000e+00                 0.000000e+00                  
16   0.000000e+00                 0.000000e+00                  
17   0.000000e+00                 0.000000e+00                  
18   5.654800e-06                 1.703010e-05                  
19   0.000000e+00                 0.000000e+00                  
20   0.000000e+00                 0.000000e+00                  
21   1.129800e-06                 2.212300e-06                  
22   0.000000e+00                 0.000000e+00                  
23   0.000000e+00                 0.000000e+00                  
24   7.619690e-05                 2.296999e-04                  
25   1.130400e-06                 1.007500e-06                  
26   0.000000e+00                 0.000000e+00                  
27   0.000000e+00                 0.000000e+00                  
28   0.000000e+00                 0.000000e+00                  
29   0.000000e+00                 0.000000e+00                  
30   0.000000e+00                 0.000000e+00                  
31   0.000000e+00                 0.000000e+00                  
32   2.542200e-06                 8.510000e-06                  
33   0.000000e+00                 0.000000e+00                  
34   2.300000e-08                 2.720000e-08                  
35   2.080480e-05                 6.265700e-05                  
36   4.170000e-08                 8.170000e-08                  
37   1.820000e-08                 1.013000e-07                  
38   0.000000e+00                 0.000000e+00                  
39   0.000000e+00                 0.000000e+00                  
40   0.000000e+00                 0.000000e+00                  
41   4.180000e-08                 1.367000e-07                  
42   1.910000e-07                 7.948000e-07                  
43   4.724000e-07                 2.281000e-07                  
44   0.000000e+00                 0.000000e+00                  
45   0.000000e+00                 0.000000e+00                  
46   6.722100e-06                 2.250270e-05                  
47   2.212000e-07                 1.027900e-06                  
48   1.096010e-05                 3.667390e-05                  
49   0.000000e+00                 0.000000e+00                  
50   0.000000e+00                 0.000000e+00                  
51   0.000000e+00                 0.000000e+00                  
52   1.172000e-07                 5.660000e-08                  
53   3.221000e-07                 7.994000e-07                  
54   1.810230e-05                 6.055380e-05                  
55   2.169140e-05                 4.952020e-05                  
56   0.000000e+00                 0.000000e+00                  
57   3.750000e-07                 1.129300e-06                  
58   3.686700e-06                 1.712860e-05                  
59   1.211100e-06                 3.817000e-06                  
60   1.520900e-06                 3.2715

In [32]:
def calculatePremium(row, layers):
    premium = 0
    for layer in layers:
        premium += ((row['buildingValue'] * row[layer + '.buildingAlr']) +
                    (row['contentsValue'] * row[layer + '.contentsAlr']) +
                    (row['businessInterruptionValue'] * row[layer + '.businessInterruptionAlr'])) * 1.5
    row['premium'] = int(premium)
    return row


final = lossCostsWithFlood.apply(calculatePremium, axis=1, layers=[
                                 'us_ws_loss_cost', 'us_cs_loss_cost', 'us_eq_loss_cost', 'us_fl_loss_cost'])

final[['id','streetAddress','cityName','admin1Code','premium']]

id                         streetAddress          cityName admin1Code  \
0    1    770 KEMP BOTTOM RD                    VICKSBURG         MS          
1    2    2001 KINGSLEY AVE                     ORANGE PARK       FL          
2    3    1050 COUNTRY LN                       ORLANDO           FL          
3    4    6675 US HIGHWAY 43                    GUIN              AL          
4    6    5775 PEACHTREE DUNWOODY RD NE         ATLANTA           GA          
5    8    8201 W BROWARD BLVD                   PLANTATION        FL          
6    9    660 W SAVANNAH CIR                    ORLANDO           FL          
7    11   1400 STATE DOCKS RD                   DECATUR           AL          
8    13   8605 LARGO LAKES DR                   LARGO             FL          
9    14   4955 MASON RD                         ATLANTA           GA          
10   15   72 MARIETTA ST NW                     ATLANTA           GA          
11   17   2450 GALLERIA PKWY SE                 ATLANTA           GA          
12   19   5637 MARINE PKWY                      NEW PORT RICHEY   FL          
13   20   13615 OLD HIGHWAY 61 N                ROBINSONVILLE     MS          
14   21   3704 LOUIS RICH RD                    NEWBERRY          SC          
15   24   5301 S CONGRESS AVE                   LAKE WORTH        FL          
16   25   300 NISSAN DR                         CANTON            MS          
17   27   1 PLASTICS DR                         LOWNDESBORO       AL          
18   29   271 WORLD DR                          ORLANDO           FL          
19   30   148 INDUSTRIAL BLVD                   EATONTON          GA          
20   33   60 HERRING RD                         NEWNAN            GA          
21   34   401 NW 42ND AVE                       PLANTATION        FL          
22   35   188 HALEY AVE BLDG 1                  SAVANNAH          GA          
23   36   100 BEST BUY DRIVE                    DUBLIN            GA          
24   39   1390 MAGIC KINGDOM DR                 ORLANDO           FL          
25   40   5300 CURETON FERRY RD                 CATAWBA           SC          
26   41   ATHENS HWY AT MINERAL SPRINGS RD      ELBERTON          GA          
27   42   3434 PEACHTREE RD NE                  ATLANTA           GA          
28   43   40 PERIMETER CTR E                    ATLANTA           GA          
29   44   3001 W DR MARTIN LUTHER KING JR BLVD  TAMPA             FL          
30   45   1567 LUKKEN INDUSTRIAL DR W           LAGRANGE          GA          
31   46   3464 MIKE PADGETT HWY                 AUGUSTA           GA          
32   47   2201 ORLEANS DR.                      ORLANDO           FL          
33   48   1400 MURPHY AVE SW                    ATLANTA           GA          
34   49   1700 CENTER ST                        MOBILE            AL          
35   50   4224 METRIC DR                        WINTER PARK       FL          
36   51   1 E TELECOM PKWY                      TAMPA             FL          
37   53   5223 ORIENT RD                        TAMPA             FL          
38   54   4000 VILLAGE VIEW DRIVE               GAINESVILLE       GA          
39   55   3900 MOTORS INDUSTRIAL WAY            ATLANTA           GA          
40   56   3393 PEACHTREE RD NE                  ATLANTA           GA          
41   59   3015 COAST LINE DR                    ORLANDO           FL          
42   60   3301 COLLEGE AVE                      DAVIE             FL          
43   63   1411 HURON ST                         JACKSONVILLE      FL          
44   64   201 MONROE ST                         MONTGOMERY        AL          
45   65   6205 PEACHTREE DUNWOODY RD NE         ATLANTA           GA          
46   66   2175 EPCOT RESORTS BLVD               ORLANDO           FL          
47   69   1700 EPCOT RESORTS BLVD               ORLANDO           FL          
48   70   4900 CAYMAN WAY                       ORLANDO           FL          
49   72   1506 BUSHY PARK RD                    GOOSE CR

# Wrap Up
Out of the 239 original locations, 92 were rejected. This was due to:
* Poor geocoding (33)
* In FEMA flood zones (22)
* High flood depth risk in RMS model (11)
* High risk scores in standalone basis or relative to other locations (26)

For the remaining 147 locations, you're confident that you've avoided the very riskiest locations, and you've calculated cat premiums for them. These premiums can be plugged into pricing formulas that take into account other sources of risk, such as crime.

# Visualization

You've been asked to provide some sense of the average cat premium by state for the 147 locations. You create a map for this.

In [29]:
# Make sure folium is available for visualization
import sys
!{sys.executable} -m pip install folium

In [30]:
visual = final[['premium', 'geocode.admin1Code']].groupby(['geocode.admin1Code']).mean()
visual.reset_index(inplace = True)
visual

geocode.admin1Code       premium
0  AL                 1263.500000 
1  FL                 61414.691176
2  GA                 106.288462  
3  MS                 78039.250000
4  SC                 26.909091

In [31]:
import folium
admin1data = "admin1-us.geojson"

m = folium.Map(location=[30.7, -84.0], zoom_start=6,
                detect_retina=True, control_scale=False)

# Create choropleth map
folium.Choropleth(
        geo_data=admin1data,
        name='choropleth',
        data=visual,
        columns=['geocode.admin1Code', 'premium'],
        key_on='feature.properties.state_code',
        fill_color='OrRd',
        nan_fill_opacity=0,
        nan_line_opacity=0,
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='Average cat premium per location'
).add_to(m)

m